<a href="https://colab.research.google.com/github/donghwi2022/ds-section1-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다음 분기에 어떤 게임을 설계해야 할까

# **Part 1. 필요 라이브러리 및 데이터 로드**
- 필요 라이브러리 import 및 한글 깨짐문제 해결
- csv 파일 load

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#한글 깨짐문제 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic');

In [ ]:
# url로 필요 csv 파일 불러오기
url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv"
df = pd.read_csv(url)
df.head()

# **Part 2. 데이터 전처리 진행**

In [ ]:
df.info()

In [ ]:
# 첫번째 column은 index를 나타내므로 해당 열 제거
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

In [ ]:
# 결측치 확인
df.isnull().sum()

In [ ]:
# 데이터 양이 충분하므로, 결측치가 존재하는 데이터 삭제
df.dropna(inplace = True)
df.info()

In [ ]:
# 중복값 확인(게임 명, 지원되는 플랫폼, 출시 연도, 장르, 게임회사 명이 같은 row는 중복데이터로 간주)
df[df.duplicated(['Name', 'Platform', 'Year', 'Genre', 'Publisher'])]

In [ ]:
# 중복되는 데이터 확인
df[(df.Name == 'Madden NFL 13') & (df.Platform == 'PS3') & (df.Year == 2012.0) & (df.Genre == 'Sports') & (df.Publisher == 'Electronic Arts')]

In [ ]:
# 중복되는 데이터 중 정확한 데이터를 알 수 없기 때문에 해당 row를 전부 삭제
duplicate_index = df[df.duplicated(['Name', 'Platform', 'Year', 'Genre', 'Publisher'])].index
duplicated_all_index = []
for i in range(len(duplicate_index)) :
    duplicated_data = df.loc[duplicate_index[i]].Name, df.loc[duplicate_index[i]].Platform, df.loc[duplicate_index[i]].Year, df.loc[duplicate_index[i]].Genre, df.loc[duplicate_index[i]].Publisher
    duplicated_all_index.extend(df[(df.Name == duplicated_data[0]) & (df.Platform == duplicated_data[1]) & (df.Year == duplicated_data[2]) & (df.Genre == duplicated_data[3]) & (df.Publisher == duplicated_data[4])].index)
    # append를 사용할 경우 배열에 배열을 추가할 시, 배열이 그대로 유지되지만 extend는 쪼개서 한 개씩 들어감
df.drop(duplicated_all_index, inplace = True)
df.info()

In [ ]:
df_1 = df.EU_Sales.str.extract(r'([^\.\d])')
df_1[0].unique()

In [ ]:
# 출고량에 해당하는 column에서 숫자가 아닌 타입을 출력
df[df.EU_Sales.str.extract(r'([^\.\d])').notnull()].head(11)